In [172]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from collections import Counter

In [173]:
train_X = np.load('./data/fashion_mnist/train_x.npy')
train_y = np.load('./data/fashion_mnist/train_y.npy')
valid_X = np.load('./data/fashion_mnist/valid_x.npy')
valid_y = np.load('./data/fashion_mnist/valid_y.npy')

test_X = np.load('./data/fashion_mnist/test_x.npy')

In [174]:
print('# of class (trian):', Counter(train_y))
print('# of class (test):',Counter(valid_y))

# of class (trian): Counter({0: 6000, 9: 5400, 8: 4200, 7: 3000, 6: 1200, 5: 600, 1: 300, 2: 300, 3: 300, 4: 300})
# of class (test): Counter({0: 100, 1: 100, 2: 100, 3: 100, 4: 100, 5: 100, 6: 100, 7: 100, 8: 100, 9: 100})


In [175]:
train_images, train_labels = train_X, train_y
valid_images, valid_labels = valid_X, valid_y

test_images = test_X

# Preprocess the dataset (normalize and convert to tensors)
train_images, valid_images = train_images / 255.0, valid_images / 255.0
test_images = test_images / 255.0

In [176]:
train_images = torch.tensor(train_images, dtype=torch.float32).reshape(-1, 28*28)
train_labels = torch.tensor(train_labels, dtype=torch.long)
valid_images = torch.tensor(valid_images, dtype=torch.float32).reshape(-1, 28*28)
valid_labels = torch.tensor(valid_labels, dtype=torch.long)

test_images = torch.Tensor(test_images).reshape(-1, 28*28)

In [177]:
# Create DataLoader for training and test sets

class_weights = [0.6, 0.3, 0.1]
targets = train_labels.tolist()

class_data_count = np.array([len(np.where(targets == t)[0]) for t in np.unique(targets)])
weight = 1.0 / class_data_count

balanced_weight = np.array([weight[t] for t in targets])
balanced_weight = torch.from_numpy(balanced_weight)
sampler = WeightedRandomSampler(balanced_weight, len(balanced_weight))

train_dataset = TensorDataset(train_images, train_labels)
valid_dataset = TensorDataset(valid_images, valid_labels)

train_loader = DataLoader(train_dataset, batch_size=64,shuffle=False, sampler=sampler)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [178]:
# Create the model
from MLP import MLP_classifier
input_size = 28*28
num_classes = 10
model = MLP_classifier(input_size, num_classes)

In [179]:
# Set up the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [180]:
# Train the model using the train method
num_epochs = 50
model.train(train_loader, valid_loader, criterion, optimizer, num_epochs)

Epoch [1/50], Loss: 1.7021
Accuracy: 76.60%
Epoch [2/50], Loss: 1.5729
Accuracy: 79.20%
Epoch [3/50], Loss: 1.5347
Accuracy: 79.70%
Epoch [4/50], Loss: 1.6165
Accuracy: 81.30%
Epoch [5/50], Loss: 1.5407
Accuracy: 80.80%
Epoch [6/50], Loss: 1.6307
Accuracy: 81.70%
Epoch [7/50], Loss: 1.6257
Accuracy: 81.30%
Epoch [8/50], Loss: 1.6250
Accuracy: 80.50%
Epoch [9/50], Loss: 1.6505
Accuracy: 79.80%
Epoch [10/50], Loss: 1.5235
Accuracy: 81.80%
Epoch [11/50], Loss: 1.5239
Accuracy: 82.20%
Epoch [12/50], Loss: 1.5706
Accuracy: 81.60%
Epoch [13/50], Loss: 1.5507
Accuracy: 83.50%
Epoch [14/50], Loss: 1.6453
Accuracy: 82.70%
Epoch [15/50], Loss: 1.5865
Accuracy: 82.50%
Epoch [16/50], Loss: 1.5540
Accuracy: 83.30%
Epoch [17/50], Loss: 1.5539
Accuracy: 81.20%
Epoch [18/50], Loss: 1.6171
Accuracy: 82.00%
Epoch [19/50], Loss: 1.6367
Accuracy: 81.40%
Epoch [20/50], Loss: 1.5491
Accuracy: 81.70%
Epoch [21/50], Loss: 1.6049
Accuracy: 82.60%
Epoch [22/50], Loss: 1.5434
Accuracy: 83.30%
Epoch [23/50], Loss

In [181]:
test_output = model.forward(test_images)
_, predicted = torch.max(test_output, 1)

In [182]:
result = pd.DataFrame({"index":np.arange(len(predicted)), "label": predicted})

In [183]:
result.to_csv("result.csv", index=None)